spark shuffle 演进的历史
* Spark 0.8及以前 Hash Based Shuffle
* Spark 0.8.1 为Hash Based Shuffle引入File Consolidation机制
* Spark 0.9 引入ExternalAppendOnlyMap
* Spark 1.1 引入Sort Based Shuffle，但默认仍为Hash Based Shuffle
* Spark 1.2 默认的Shuffle方式改为Sort Based Shuffle
* Spark 1.4 引入Tungsten-Sort Based Shuffle
* Spark 1.6 Tungsten-sort并入Sort Based Shuffle
* Spark 2.0 Hash Based Shuffle退出历史舞台

### 一. Shuffle过程

目前版本的shuffle, 都是使用排序相关的shuffle; 整体上spark shuffle分为shuffle read和shuffle write:   
* **shuffle write阶段**   
  大体上经过排序, 聚合, 归并(多个文件spill磁盘的情况), 最终, 每个task生成2种文件: 数据文件和索引文件. 
    * 数据文件: 相同分区的数据在文件内是连续的, 
    * 索引文件: 记录了每个分区的开始位置和结束位置  
    
    
* **shuffle read阶段**   
   首先通过网络从各个writer节点获取给定分区的数据, 即数据文件中某一连续区域的数据, 然后再将这些不同节点下的分区文件进行排序, 合并; 最终形成分区下的结果
   
### 二. 三种shuffle writer的过程 
#### 1. BypassMergeSortShuffleWriter   
* BypassMergeSortShuffleWriter实现过程   
  因为, BypassMergeSortShuffleWriter大体实现过程是: 首先为每个分区创建一个分区文件, 将对应的数据写入到各自的分区文件中; 最终合并所有的分区文件形成一个大文件, 且伴随产生一个索引文件记录分区的开始位置和结束位置. 因为这种shuffle不需要进行排序和combine等操作, 因此不会十分消耗内存.
  
       
* BypassMergeSortShuffleWriter的触发条件有2个: 
    1. map task的数量不多: 小于`spark.shuffle.sort.bypassMergeThreshold`值, 默认为200
    2. 非map端聚合的算子: 比如`reduceByKey`就不能使用bypasssertshuffle.
    SortShuffleWriter
    
#### 2. SortShuffleWriter
SortShuffleWriter是日常使用最频繁的shuffle过程; SortShuffleWriter主要使用`ExternalSorter`对数据进行排序, 合并, 聚合(combine). 最后产生数据文件和索引文件
* **首先**, 根据算子是否需要map端进行combine, 数据会被写入到不同的数据结构: 
    1. 需要进行combine的算子, 数据写入到`PartitionedAppendOnlyMap`这个map中
    2. 不需要进行combine的算子, 数据写入到`PartitionedPairBuffer`这个数组中
* **然后**, 每个一段时间, 向MemoryManager申请内存
    * 如果可以一直申请到内存, 则`PartitionedAppendOnlyMap`和`PartitionedPairBuffer`中的数据会一直存在于内存中, 可见这两个数据结构很消耗内存.
    * 如果发现申请不到了, 则会把内存中的数据spill到磁盘文件; 在spill到文件之前, 需要先把`PartitionedAppendOnlyMap`和`PartitionedPairBuffer`中的数据和已经spill到磁盘的数据进行合并(使用归并排序合并). `PartitionedAppendOnlyMap`和`PartitionedPairBuffer`内存中的排序算法使用[TimSort]()算法  
* **最后**, 写入文件一般是先把数据写入一个buffer, 再把buffer中的数据写入到文件. 此过程受2个参数控制  
    * [spark.shuffle.file.buffer](#param1)
    * spark.shuffle.spill.batchSize 
    
    
#### 3. UnsafeShuffleWriter
`UnsafeShuffleWriter`是`SortShuffleWriter`的优化版本, 大体上相差不大, 从内存角度看, 主要差异为: 
* 一方面，在 SortShuffleWriter 的 PartitionedAppendOnlyMap 或者 PartitionedPairBuffer 中，存储的是键值或者值的具体类型，也就是 Java 对象，是反序列化过后的数据。而在 UnsafeShuffleWriter 的 ShuffleExternalSorter 中数据是序列化以后存储到实际的 Page 中，而且在写入数据过程中会额外写入长度信息。总体而言，序列化以后数据大小是远远小于序列化之前的数据。

* 另一方面，UnsafeShuffleWriter 中需要额外的存储记录（LongArray），它保存着分区信息和实际指向序列化后数据的指针（经过编码的Page num 以及 Offset）。相对于 SortShuffleWriter， UnsafeShuffleWriter 中这部分存储的开销是额外的。

### 三. Shuffle Read的过程

#### 1. 数据获取阶段
数据获取分为本地获取和远端获取: 
* 本地获取: 
   直接从本地的BlockManager获取数据
* 远端获取:  远端获取数据需要走网络, 有一些参数可以控制
    * `spark.reducer.maxSizeInFlight` (默认 48M): 接收buffer的缓冲区大小
    * `spark.reducer.maxReqsInFlight`:   
       限制同时能有多少个远程机器进行拉取本机器文件的请求.随着集群增大，需要对此做出限制。否则可能会使本机负载过大而挂掉。

#### 2. 拉取的数据进行spill, combine, 合并文件   
   shuffle reader的溢写, 排序和合并文件操作, 与shuffle writer相似, 使用ExternalAppendOnlyMap完成

### 四. 什么情况下可能会导致Spark Shuffle阶段发生OOM
* **1. 并发度过高**  
 可能1个Execotor上设置的并发任务书过多 导致每个job分到的内存太少
 
* **2. 数据倾斜时, 单个文件过大, 导致拉取到内存后发生OOM**  
 在 Reduce 获取数据时，由于数据倾斜，有可能造成单个 Block 的数据非常的大，默认情况下是需要有足够的内存来保存单个 Block 的数据。因此，此时极有可能因为数据倾斜造成 OOM。 可以设置 spark.maxRemoteBlockSizeFetchToMem 参数，设置这个参数以后，超过一定的阈值，会自动将数据 Spill 到磁盘，此时便可以避免因为数据倾斜造成 OOM 的情况。在我们的生产环境中也验证了这点，在设置这个参数到合理的阈值后，生产环境任务 OOM 的情况大大减少了。
 
* **3. spill不及时**  
Spark 目前采用的是抽样统计的方式来计算 MemoryConsumer 已经使用的内存，从而造成堆内内存的实际使用量不是特别准确。从而有可能因为不能及时 Spill 而导致 OOM。